In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession 
from pyspark import SparkConf,SparkContext
from pyspark.sql.types import IntegerType, StringType
#from pyspark.sql.functions import udf
import pyspark.sql.functions as funct
import os

#conf = SparkConf()
#conf.setMaster('local')
#conf.setAppName('pyspark-shell')
#conf.set('spark.driver.host', '127.0.0.1')
#os.environ['PYSPARK_PYTHON'] = '/Users/ccompain/.pyenv/versions/miniconda3-latest/bin/python' # Needs to be explicitly provided as env. Otherwise workers run Python 2.7
#os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'

#sc = SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

#sc.setLogLevel("WARN") 

In [2]:
r=spark.createDataFrame([("A","B"),("B","C"),("B","D"),("D","E"),("F","G"),("G","H")]).toDF("N1","N2") 
#r.printSchema()
r.show()
#r.collect()

+---+---+
| N1| N2|
+---+---+
|  A|  B|
|  B|  C|
|  B|  D|
|  D|  E|
|  F|  G|
|  G|  H|
+---+---+



In [3]:
log4jLogger = spark._jvm.org.apache.log4j
LOGGER = log4jLogger.LogManager.getLogger(__name__)


In [4]:
new_pair_flag = True
iteration = 0
dedupJob = r

In [5]:
while new_pair_flag:
    iteration += 1
    newPair = False
    newPairs = 0

    # CCF-iterate (MAP)
    mapJob = dedupJob.union(dedupJob.select('N2', 'N1')).persist()
  #  print("Map "+str(iteration))
  #  print(mapJob.show())

    # CCF-iterate (REDUCE)
    minDF=mapJob.groupBy('N1').agg(funct.min(mapJob['N2']).alias('minN')).where('min(N2)<N1').persist()
  #  print("minDF "+str(iteration))
  #  print(minDF.show())
    supplDF=mapJob.join(minDF, "N1").where("minN<>N2").select('N2','minN').withColumnRenamed('N2','N1').withColumnRenamed('minN','N2').persist()
  #  print("supplDF "+str(iteration))
  #  print(supplDF.show())
    newPairs=supplDF.count()
    reduceJob = supplDF.union(minDF).persist()
  #  print("reduceJob "+str(iteration))
  #  print(reduceJob.show())
    
    # CCF-dedup 
    dedupJob = reduceJob.distinct().persist() #.orderBy("N1", "N2")
  #  print("dedupJob "+str(iteration))
  #  print(dedupJob.show())

    # Force the RDD evalusation
#    tmp = dedupJob.count()

    # Prepare next iteration
    #mapJob = dedupJob
    new_pair_flag = bool(newPairs)

    #LOGGER.warn("Iteration :"+str(iteration)+" --- "+" newPair : "+str(new_pair_flag))
    print("Iteration :"+str(iteration)+" --- "+" newPairs : "+str(newPairs))


Iteration :1 ---  newPair : 4
Iteration :2 ---  newPair : 9
Iteration :3 ---  newPair : 4
Iteration :4 ---  newPair : 0


In [8]:
#count1 = funct.udf(lambda s: s+1, IntegerType())
#_ = spark.udf.register("count1", count1)

result= dedupJob.groupBy('N2').count().orderBy('count',ascending=False) #.withColumn('nbComponents',count1('count')).select('N2','NbComponents').orderBy('NbComponents',ascending=False)
print(range(result.show()))


+---+-----+
| N2|count|
+---+-----+
|  A|    4|
|  F|    2|
+---+-----+



TypeError: 'NoneType' object cannot be interpreted as an integer